# Exercise 02

In [82]:
import pandas as pd
import sqlite3

## 1. Create a connection to the database using the library sqlite3

In [83]:
connection = sqlite3.connect('../data/checking-logs.sqlite')
cursor = connection.cursor()

## 2. Create a new table datamart in the database by joining the tables pageviews and checker using only one query

- the table should have the following columns: uid, labname, first_commit_ts, first_view_ts
- first_commit_ts is just a new name of the column timestamp from the checker table, it shows the first commit from a particular lab and from a particular user
- first_view_ts is the first visit of a user to the table pageviews, timestamp when a user visited the newsfeed
- status = ’ready’ should still be a filter
- numTrials = 1 should still be a filter
- labnames should still be from the list: ’laba04’, ’laba04s’, ’laba05’, ’laba06’, ’laba06s’, ’project1’
- the table should contain only the users (uids with user_*) and not the admins
- first_commit_ts and first_view_ts should be parsed as datetime64[ns]

In [84]:
sql_query = '''
CREATE TABLE IF NOT EXISTS datamart AS
SELECT checker.uid, labname, checker.timestamp AS first_commit_ts, pageviews.datetime AS first_view_ts
FROM checker
FULL JOIN pageviews ON checker.uid = pageviews.uid 
WHERE status = 'ready'
AND numTrials = 1
AND labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
AND checker.uid LIKE 'user_%'
GROUP BY first_commit_ts
'''

cursor.execute(sql_query)
connection.commit()


In [85]:
sql_query = '''SELECT * FROM datamart'''

df = pd.read_sql(sql_query, connection)
df['first_commit_ts'] = pd.to_datetime(df['first_commit_ts'])
df['first_view_ts'] = pd.to_datetime(df['first_view_ts'])

df.dtypes

uid                        object
labname                    object
first_commit_ts    datetime64[ns]
first_view_ts      datetime64[ns]
dtype: object

## 3. Using Pandas methods, create two dataframes: test and control

- test should have the users that have the values in first_view_ts
- control should have the users that have missing values in first_view_ts
- replace the missing values in the control with the average first_view_ts of the test users, we will use this value for the future analysis
- save both tables into the database, you will use them in the next exercises

In [86]:
test = df.dropna()
control = df[df.isna().any(axis=1)]
mean_value = control['first_view_ts'].fillna(value=test['first_view_ts'].mean())
control.loc[control['first_view_ts'].isna(), 'first_view_ts'] = mean_value
control

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,2020-04-27 00:40:05.761783552
1,user_4,laba04,2020-04-17 11:33:17.366400,2020-04-27 00:40:05.761783552
2,user_4,laba04s,2020-04-17 11:48:41.992466,2020-04-27 00:40:05.761783552
5,user_2,laba04,2020-04-18 13:42:35.482008,2020-04-27 00:40:05.761783552
6,user_2,laba04s,2020-04-18 13:51:22.291271,2020-04-27 00:40:05.761783552
...,...,...,...,...
126,user_2,laba06s,2020-05-19 14:45:03.908268,2020-04-27 00:40:05.761783552
132,user_6,laba06s,2020-05-20 14:50:07.609937,2020-04-27 00:40:05.761783552
134,user_7,laba06s,2020-05-20 23:05:37.742597,2020-04-27 00:40:05.761783552
135,user_23,laba06,2020-05-21 08:34:10.517205,2020-04-27 00:40:05.761783552


In [87]:
test.to_sql('test', con=connection, if_exists='replace', index=False)
control.to_sql('control', con=connection, if_exists='replace', index=False)

81

## 4. Close the connection

In [88]:
connection.close()